# F-test

#### Imports

In [7]:
'''
Credit to GitHub user Jaimin09
Link: https://github.com/Jaimin09/Coding-Lane-Assets/tree/main/Logistic%20Regression%20in%20Python%20from%20Scratch
Last accessed: 28/10/2021
'''
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import portablelogresmodel as model
import scipy

from scipy import stats

# ! Get dataset
filepath = 'Datasets\dec_sep_MPHWA.csv'
df = pd.read_csv(filepath)
df= df.reset_index()

dec_path = 'Datasets\dec_MPHWA.csv'
dec_df = pd.read_csv(dec_path)
dec_df = dec_df.reset_index()

# Execute model

In [8]:
# ! Set seed and seed calling function
rng = np.random.default_rng(1)

X_list = ['ID',  
        'PreviousMedals',
        'Height_div_avg', 
        'Weight_div_avg', 
        'Age_div_avg'
        ]

Y_list = ['ID', 'MedalEarned']

# Import model parameters
W_array = np.genfromtxt('Parameters/W.csv', delimiter=',')
B_array = np.genfromtxt('Parameters/B.csv', delimiter=',')

# F-test

# Bearbejd data 

In [9]:
# Produce weight for model
def ProduceWeights(df, W, B):

    # Get data for model
    X_model_df = df[X_list]
    Y_model_df = df[Y_list]

    X_, Y_ = model.Reshape(X_model_df, Y_model_df)
    
    # Get model guesses 
    lin_func = np.dot(W.T, X_) + B # Linear function
    sf = model.Sigmoid(lin_func) # Sigmoid function
    
    return sf


In [10]:
def f_test_loop(W, B):
    sig0_store = []
    sig1_store = []
    drop_ID = X_list[1:]
    for i,element in enumerate(drop_ID):
        
        # Change first parameter to 0 
        print(f'vores i er {i}')
        print(f'vores element er {element}')
        W1 = copy.deepcopy(W)
        print(f'vores len er {len(W1)}')
        W1[i] = 0
        
        # Get list of model guesses if weights are unchanged 
        sig_0 = ProduceWeights(df, W, B)
        print(sig_0)
        
        # Get a list of model guesses where weights are changed
        sig_1 = ProduceWeights(df, W1, B)
        print(sig_1)
        
        sig0_store.append(sig_0)
        sig1_store.append(sig_1)
    return sig0_store, sig1_store



# Call F_test

In [11]:
W_par = np.array([W_array[0][0], W_array[1][0], W_array[2][0], W_array[3][0]], ndmin= 0)
B_par = B_array[0]

p_vector_0, p_vector_1 = f_test_loop(W_par,B_par) 

vores i er 0
vores element er PreviousMedals
vores len er 4
[0.92524216 0.20173657 0.43566097 ... 0.55253714 0.37581016 0.29995489]
[0.92524216 0.20173657 0.43566097 ... 0.55253714 0.37581016 0.29995489]
vores i er 1
vores element er Height_div_avg
vores len er 4
[0.92524216 0.20173657 0.43566097 ... 0.55253714 0.37581016 0.29995489]
[0.92213526 0.19923577 0.41987367 ... 0.55507281 0.36356257 0.31676768]
vores i er 2
vores element er Weight_div_avg
vores len er 4
[0.92524216 0.20173657 0.43566097 ... 0.55253714 0.37581016 0.29995489]
[0.32566492 0.33365407 0.46003613 ... 0.46465944 0.4624237  0.44020105]
vores i er 3
vores element er Age_div_avg
vores len er 4
[0.92524216 0.20173657 0.43566097 ... 0.55253714 0.37581016 0.29995489]
[0.94741978 0.25644853 0.39402305 ... 0.48648425 0.33168873 0.25306099]


# Test for normality

In [12]:
def normality_loop(x_1, y_1):
    #print(f'vores p_vector_0 er {p_vector_0} \n vores p_vector_1 er {p_vector_1}')
    x_norm = []
    y_norm = []
    drop_ID = X_list[1:]
    for i in range(len(x_1)):
        x_norm.append(scipy.stats.shapiro(x_1[i]))
        y_norm.append(scipy.stats.shapiro(y_1[i]))
        print(f' vores variabel er: {drop_ID[i]} \n vores x_norm er {x_norm[i]}, \n vores y_norm er {y_norm[i]}')
    return x_norm,y_norm
shapiro_0, shapiro_1 = normality_loop(p_vector_0, p_vector_1)



#z = scipy.stats.probplot(df_sig_0["Prob"], dist = 'norm', plot = plt)
#q = scipy.stats.probplot(df_sig_1["Prob"], dist = 'norm', plot = plt)

#plt.show(z)
#plt.show(q)

# Test for normality showed that including previous medals, caused the distribution to be non-normal. PreviousMedals is therefore excluded. 

 vores variabel er: PreviousMedals 
 vores x_norm er ShapiroResult(statistic=0.8946238160133362, pvalue=0.0), 
 vores y_norm er ShapiroResult(statistic=0.8800283074378967, pvalue=0.0)
 vores variabel er: Height_div_avg 
 vores x_norm er ShapiroResult(statistic=0.8946238160133362, pvalue=0.0), 
 vores y_norm er ShapiroResult(statistic=0.8890291452407837, pvalue=0.0)
 vores variabel er: Weight_div_avg 
 vores x_norm er ShapiroResult(statistic=0.8946238160133362, pvalue=0.0), 
 vores y_norm er ShapiroResult(statistic=0.8226956129074097, pvalue=0.0)
 vores variabel er: Age_div_avg 
 vores x_norm er ShapiroResult(statistic=0.8946238160133362, pvalue=0.0), 
 vores y_norm er ShapiroResult(statistic=0.8803070187568665, pvalue=0.0)


# F-test

In [13]:
# F-test: https://link.springer.com/book/10.1007%2F978-3-319-46162-5 
# Code adapted from: https://www.statology.org/f-test-python/
#define F-test function

def f_test(x,y):
    f_test_list = []
    drop_ID = X_list[1:]
    for i in range(len(x)):
        f = np.var(x[i], ddof=1)/np.var(y[i], ddof=1) #calculate F test statistic 
        dfn = x[i].size-1 #define degrees of freedom numerator 
        dfd = y[i].size-1 #define degrees of freedom denominator 
        p = 1-scipy.stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic
        f_test_list.append(p)
        print(f'vores variabel er {drop_ID[i]} \n vores p er {p}')
        if p <= 0.05:
            print(f'nulhypotesen forkastes da p-værdien = {p}. Dermed er sandsynligheden for at de to fordelinger har samme varians lav. Variablen har stor betydning for modellens output.')
        else:
            print(f'nulhypotesen kan ikke forkastes da p-værdien = {p}. Dermed er der høj sandsynlighed for at de to fordelinger har samme varians. Variablen kan have stor betydning for modellens output.')
    return f_test_list
f_test(p_vector_0, p_vector_1)





vores variabel er PreviousMedals 
 vores p er 5.223256799169462e-05
nulhypotesen forkastes da p-værdien = 5.223256799169462e-05. Dermed er sandsynligheden for at de to fordelinger har samme varians lav. Variablen har stor betydning for modellens output.
vores variabel er Height_div_avg 
 vores p er 0.02825330415181948
nulhypotesen forkastes da p-værdien = 0.02825330415181948. Dermed er sandsynligheden for at de to fordelinger har samme varians lav. Variablen har stor betydning for modellens output.
vores variabel er Weight_div_avg 
 vores p er 1.1102230246251565e-16
nulhypotesen forkastes da p-værdien = 1.1102230246251565e-16. Dermed er sandsynligheden for at de to fordelinger har samme varians lav. Variablen har stor betydning for modellens output.
vores variabel er Age_div_avg 
 vores p er 0.9737840882713829
nulhypotesen kan ikke forkastes da p-værdien = 0.9737840882713829. Dermed er der høj sandsynlighed for at de to fordelinger har samme varians. Variablen kan have stor betydning f

[5.223256799169462e-05,
 0.02825330415181948,
 1.1102230246251565e-16,
 0.9737840882713829]